In [233]:
import pandas as pd

# Load CSV files
match_results_df = pd.read_csv('csv/wedstrijden_onverwerkd.csv')
goal_events_df = pd.read_csv('csv/doelpunten_onverwerkd.csv')
standings_df = pd.read_csv('csv/klassement_onverwerkd.csv')

goal_events_df = goal_events_df[goal_events_df['valid_goal'] == True]
# Filter of vervang niet-numerieke waarden
match_results_df['doelpunten_thuisploeg'] = pd.to_numeric(match_results_df['doelpunten_thuisploeg'], errors='coerce').fillna(0)
match_results_df['doelpunten_uitploeg'] = pd.to_numeric(match_results_df['doelpunten_uitploeg'], errors='coerce').fillna(0)

# Nu kunt u de conversie naar int veilig uitvoeren
match_results_df['doelpunten_thuisploeg'] = match_results_df['doelpunten_thuisploeg'].astype(int)
match_results_df['doelpunten_uitploeg'] = match_results_df['doelpunten_uitploeg'].astype(int)

In [234]:
# Vind alle verplaatste matches, namelijk de match id zal dezelfde zijn als de vorige
match_results_df['same_as_previous_match'] = match_results_df['id_match'].eq(match_results_df['id_match'].shift())

# Selecteer alleen de rijen waar 'same_as_previous_match' True is
duplicate_matches = match_results_df[match_results_df['same_as_previous_match']]
duplicate_matches.head(30)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match


In [235]:
# Bereken goals
goals_home_team = goal_events_df[goal_events_df['goal_team_stamnummer'] == goal_events_df['thuisploeg_stamnummer']].groupby('id_match').size()
goals_away_team = goal_events_df[goal_events_df['goal_team_stamnummer'] == goal_events_df['uitploeg_stamnummer']].groupby('id_match').size()

# Omzetting
goals_home_team_df = goals_home_team.reset_index(name='calculated_doelpunten_thuisploeg')
goals_away_team_df = goals_away_team.reset_index(name='calculated_doelpunten_uitploeg')

# Vervang NaN values in goal kolom met 0
merged_df = pd.merge(match_results_df, goals_home_team_df, on='id_match', how='outer')
merged_df = pd.merge(merged_df, goals_away_team_df, on='id_match', how='outer')
merged_df['calculated_doelpunten_thuisploeg'] = merged_df['calculated_doelpunten_thuisploeg'].fillna(0).astype(int)
merged_df['calculated_doelpunten_uitploeg'] = merged_df['calculated_doelpunten_uitploeg'].fillna(0).astype(int)

# Bereken verschil tussen berekening en resultaat
discrepancies_goals = merged_df[
    ((merged_df['calculated_doelpunten_thuisploeg'] != merged_df['doelpunten_thuisploeg']) |
    (merged_df['calculated_doelpunten_uitploeg'] != merged_df['doelpunten_uitploeg'])) &
    ~((merged_df['doelpunten_thuisploeg'] == 5) & (merged_df['doelpunten_uitploeg'] == 0)) &
    ~((merged_df['doelpunten_thuisploeg'] == 0) & (merged_df['doelpunten_uitploeg'] == 5))
]

# Select only relevant columns for display - Lommel 2002, Excelsior 2009 verwijderen we alles van, matches, klassement en doelpunten
discrepancies_goals.head(1000)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match,calculated_doelpunten_thuisploeg,calculated_doelpunten_uitploeg
13329,1977,14,1977/11/20,15:00:00,4012716,19,KV Kortrijk,4451,KSV Waregem,1,1,False,0,0
13330,1977,15,1977/11/26,19:30:00,4012718,22,R Charleroi SC,35,RSC Anderlecht,2,1,False,0,0
13335,1977,15,1977/11/27,15:00:00,4012722,2300,KSK Beveren,4,RFC Luik,2,0,False,0,0
13337,1977,15,1977/11/27,15:00:00,4012724,30,Lierse SK,19,KV Kortrijk,2,0,False,0,0
13340,1977,16,1977/12/03,20:00:00,4012726,35,RSC Anderlecht,2300,KSK Beveren,0,1,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15610,1969,15,1969/12/14,14:30:00,4010466,16,Standard Luik,4451,KSV Waregem,2,0,False,0,0
15611,1969,15,1969/12/14,14:30:00,4010467,35,RSC Anderlecht,7,KAA Gent,3,1,False,0,0
15612,1969,15,1969/12/14,14:30:00,4010468,373,Sint-Truidense VV,13,Beerschot,3,0,False,0,0
15613,1969,15,1969/12/14,14:30:00,4010469,2300,KSK Beveren,522,Beringen FC,2,1,False,0,0


In [236]:
#Controle voor (5:0 of 0:5's)
discrepancies_goals = merged_df[
    (merged_df['calculated_doelpunten_thuisploeg'] != merged_df['doelpunten_thuisploeg']) |
    (merged_df['calculated_doelpunten_uitploeg'] != merged_df['doelpunten_uitploeg'])
]
discrepancies_goals.head(1000)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match,calculated_doelpunten_thuisploeg,calculated_doelpunten_uitploeg
356,2022,14,2022/10/23,18:30:00,3851182,16,Standard Luik,35,RSC Anderlecht,5,0,False,3,1
380,2022,17,2022/11/12,20:45:00,3851244,22,R Charleroi SC,25,KV Mechelen,0,5,False,1,0
728,2021,22,2022/01/15,16:15:00,3598295,18,Oud-Heverlee Leuven,25,KV Mechelen,5,0,False,0,0
841,2021,34,2022/04/10,18:30:00,3598351,10,Union Saint-Gilloise,13,Beerschot,5,0,False,0,0
2427,2014,11,2014/10/19,14:30:00,2503727,16,Standard Luik,5381,SV Zulte Waregem,0,5,False,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15573,1969,10,1969/11/09,15:00:00,4010432,10,Union Saint-Gilloise,22,R Charleroi SC,2,0,False,0,0
15574,1969,10,1969/11/09,15:00:00,4010433,53,AS Oostende KM,4,RFC Luik,2,2,False,0,0
15576,1969,10,1969/11/11,15:00:00,4010435,373,Sint-Truidense VV,3,Club Brugge,2,2,False,0,0
15577,1969,10,1970/03/22,16:00:00,4010565,30,Lierse SK,35,RSC Anderlecht,1,0,False,0,0


In [237]:
# controle van aantal_wedstrijden
# Geen enkel record met meer wedstrijden dan speeldagen
number_of_games_check = standings_df[standings_df.speeldag < standings_df.aantal_wedstrijden]
number_of_games_check.head(30)

,seizoen,speeldag,stand,naam_ploeg,id_ploeg,aantal_wedstrijden,aantal_gewonnen,aantal_gelijk,aantal_verloren,doelpunten_voor,doelpunten_tegen,verschil,punten


In [238]:
# controle van aantal_wedstrijden
# wel records met minder wedstrijden dan speeldagen, maar bij nazicht van een aantal records klopt dit
number_of_games_check = standings_df[standings_df.speeldag > standings_df.aantal_wedstrijden]
number_of_games_check.seizoen.value_counts()

1975    551
2002    442
2009    360
Name: seizoen, dtype: int64

In [239]:
# geen records met standings_df.Played != (standings_df.Wins + standings_df.Losses + standings_df.Draws
number_of_games_check = standings_df[standings_df.aantal_wedstrijden != (standings_df.aantal_gewonnen + standings_df.aantal_gelijk + standings_df.aantal_verloren)]
number_of_games_check.count()

seizoen               0
speeldag              0
stand                 0
naam_ploeg            0
id_ploeg              0
aantal_wedstrijden    0
aantal_gewonnen       0
aantal_gelijk         0
aantal_verloren       0
doelpunten_voor       0
doelpunten_tegen      0
verschil              0
punten                0
dtype: int64

In [240]:
# geen records met (uitslagen.doelpunten_voor - uitslagen.doelpunten_tegen) != (uitslagen.verschil)
standings_df['calculated_goal_verschil'] = standings_df['doelpunten_voor'] - standings_df['doelpunten_tegen']
controle_aantal_wedstrijden = standings_df[standings_df['calculated_goal_verschil'] != standings_df['verschil']]
controle_aantal_wedstrijden.count()

seizoen                     0
speeldag                    0
stand                       0
naam_ploeg                  0
id_ploeg                    0
aantal_wedstrijden          0
aantal_gewonnen             0
aantal_gelijk               0
aantal_verloren             0
doelpunten_voor             0
doelpunten_tegen            0
verschil                    0
punten                      0
calculated_goal_verschil    0
dtype: int64

In [241]:
# het totaal aantal gewonnen matchen per seizoen en per ploeg
points_df = standings_df.groupby(['seizoen','speeldag','id_ploeg', 'punten'])[['aantal_gewonnen','aantal_gelijk','aantal_verloren']].sum()
points_df = points_df.reset_index()
points_df = points_df.sort_values(['seizoen','speeldag','aantal_gewonnen','aantal_gelijk','aantal_verloren'], ascending = [True, True, False, False, False])

In [242]:
points_df['punten_check'] = points_df['aantal_gewonnen'] * 3 + points_df['aantal_gelijk'] + points_df['aantal_verloren'] * 0
discrepancies_df = points_df[points_df['punten'] != points_df['punten_check']]
discrepancies_df.head(100)

,seizoen,speeldag,id_ploeg,punten,aantal_gewonnen,aantal_gelijk,aantal_verloren,punten_check
1,1960,1,2,2:0,1,0,0,3
3,1960,1,4,2:0,1,0,0,3
5,1960,1,10,2:0,1,0,0,3
6,1960,1,13,2:0,1,0,0,3
7,1960,1,16,2:0,1,0,0,3
...,...,...,...,...,...,...,...,...
95,1960,6,3434,2:10,0,2,4,2
103,1960,7,16,12:2,5,2,0,17
97,1960,7,2,10:4,4,2,1,14
99,1960,7,4,10:4,4,2,1,14


In [243]:
match_results_df['datum'] = pd.to_datetime(match_results_df['datum'])
goal_events_df['datum'] = pd.to_datetime(goal_events_df['datum'])

merged_df = pd.merge(goal_events_df, match_results_df, on='id_match', suffixes=('_goal', '_match'))

# Check for discrepancies in dates
date_discrepancies_df = merged_df[merged_df['datum_goal'] != merged_df['datum_match']]
date_discrepancies_df.head(30)

,seizoen_goal,dag,datum_goal,tijd_goal,id_match,thuisploeg_stamnummer_goal,thuisploeg_goal,uitploeg_goal,uitploeg_stamnummer_goal,goal_time,...,speeldag,datum_match,tijd_match,thuisploeg_stamnummer_match,thuisploeg_match,uitploeg_stamnummer_match,uitploeg_match,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match


In [244]:
# Groepeer de gegevens op 'Day' en controleer de datums
sorted_dates = match_results_df.sort_values(by=['speeldag', 'datum'])
date_discrepancies_df = sorted_dates.groupby('speeldag')['datum'].apply(lambda x: x.is_monotonic_increasing)

date_discrepancies_df = date_discrepancies_df[date_discrepancies_df == False]
date_discrepancies_df.head(30)


Series([], Name: datum, dtype: bool)

In [245]:
# Sorting the data by season, day, and date
sorted_dates = match_results_df.sort_values(by=['seizoen', 'speeldag', 'datum'])

# Group by 'season' and 'day', and check if dates are in chronological order within each group
grouped = sorted_dates.groupby(['seizoen', 'speeldag'])
date_order_check = grouped['datum'].apply(lambda x: x.is_monotonic_increasing)

# Identifying seasons and days where dates are not in chronological order
date_discrepancies = date_order_check[date_order_check == False]
date_discrepancies.head(30)

Series([], Name: datum, dtype: bool)

In [246]:
# Filter matches that ended 0-0
matches_0_0 = match_results_df[(match_results_df['doelpunten_thuisploeg'] == 0) & (match_results_df['doelpunten_uitploeg'] == 0)]

# Get match_ids of 0-0 matches
match_ids_0_0 = matches_0_0['id_match']

# Check if these matches have any goals in goal_events.csv
goals_in_0_0_matches = goal_events_df[goal_events_df['id_match'].isin(match_ids_0_0)]
goals_in_0_0_matches.head(100)

,seizoen,dag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg,uitploeg_stamnummer,goal_time,real_time_goal,goal_team_naam,goal_team_stamnummer,stand_thuis,stand_uit,valid_goal


In [247]:
# Filter op een specifiek seizoen en speeldag
filtered_df = standings_df[(standings_df['seizoen'] == 2020)]


# Sorting the DataFrame
sorted_df = filtered_df.sort_values(
    ['seizoen', 'speeldag', 'punten', 'aantal_gewonnen', 'calculated_goal_verschil', 'doelpunten_voor'], 
    ascending=[True, True, False, False, False, False]
)

# Calculate rank based on the sorted DataFrame
sorted_df['calculated_stand'] = sorted_df.groupby(['seizoen', 'speeldag']).cumcount() + 1

# Adjust for ties in ranking
sorted_df['calculated_stand'] = sorted_df.groupby(
    ['seizoen', 'speeldag', 'punten', 'aantal_gewonnen', 'calculated_goal_verschil', 'doelpunten_voor']
)['calculated_stand'].transform('min')

# Merging calculated ranks back to the original DataFrame
standings_df = standings_df.merge(
    sorted_df[['seizoen', 'speeldag', 'id_ploeg', 'calculated_stand']], 
    on=['seizoen', 'speeldag', 'id_ploeg'], 
    how='left'
)

# Checking for discrepancies
discrepancies_standing = standings_df[
    (standings_df['seizoen'] == 2020) & (standings_df['speeldag'] == 4) & 
    (standings_df['stand'] != standings_df['calculated_stand'])
]

discrepancies_standing.head(1000)


,seizoen,speeldag,stand,naam_ploeg,id_ploeg,aantal_wedstrijden,aantal_gewonnen,aantal_gelijk,aantal_verloren,doelpunten_voor,doelpunten_tegen,verschil,punten,calculated_goal_verschil,calculated_stand
1742,2020,4,1,R Charleroi SC,22,4,4,0,0,7,1,6,12,6,17.0
1743,2020,4,2,Standard Luik,16,4,3,1,0,6,1,5,10,5,18.0
1744,2020,4,3,Beerschot V.A.,13,4,3,0,1,6,5,1,9,1,1.0
1745,2020,4,4,KV Kortrijk,19,4,2,1,1,4,4,0,7,0,2.0
1746,2020,4,5,Club Brugge,3,4,2,0,2,6,3,3,6,3,3.0
1747,2020,4,6,Zulte Waregem,5381,4,2,0,2,7,6,1,6,1,4.0
1748,2020,4,7,Cercle Brugge,12,4,2,0,2,5,5,0,6,0,5.0
1749,2020,4,8,RSC Anderlecht,35,4,1,3,0,8,6,2,6,2,6.0
1750,2020,4,9,OH Leuven,18,4,1,2,1,6,6,0,5,0,7.0
1751,2020,4,10,KRC Genk,322,4,1,2,1,4,4,0,5,0,8.0
